In [2]:
import numpy as np
import pandas as pd
import warnings

In [3]:
# collect the data
df_agent_report = pd.read_excel("Agent_Login_Report.xls",header=2)
df_agent_performance = pd.read_excel("AgentPerformance.xlsx",header=1)

C:\Users\DELL\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
print(df_agent_report.shape)
df_agent_report.head()

(1000, 6)


,SL No,Agent,Date,Login Time,Logout Time,Duration
0,1,Shivananda Sonwane,30-Jul-22,03:35:29 PM,05:39:39 PM,02:04:10
1,2,Khushboo Priya,30-Jul-22,03:06:59 PM,03:07:16 PM,00:00:17
2,3,Nandani Gupta,30-Jul-22,03:04:24 PM,05:31:07 PM,02:26:42
3,4,Hrisikesh Neogi,30-Jul-22,02:34:29 PM,03:19:35 PM,00:45:06
4,5,Mukesh,30-Jul-22,02:03:15 PM,03:11:52 PM,01:08:36


In [5]:
print(df_agent_performance.shape)
df_agent_performance.head()

(2160, 8)


,SL No,Date,Agent Name,Total Chats,Average Response Time,Average Resolution Time,Average Rating,Total Feedback
0,1,2022-07-30,Prerna Singh,11,00:00:38,00:04:20,4.11,9
1,2,2022-07-30,Nandani Gupta,11,00:01:15,00:28:25,3.14,7
2,3,2022-07-30,Ameya Jain,14,00:00:30,00:11:36,4.55,11
3,4,2022-07-30,Mahesh Sarade,14,00:01:04,00:15:46,4.71,7
4,5,2022-07-30,Swati,14,00:01:11,00:16:33,3.67,6


### 1 .Find out there avarage rating on weekly basis keep this in a mind that they take two days of leave


In [6]:
df_agent_performance['Date']=pd.to_datetime(df_agent_performance['Date'])

In [7]:
df_agent_performance.dtypes

SL No                               int64
Date                       datetime64[ns]
Agent Name                         object
Total Chats                         int64
Average Response Time              object
Average Resolution Time            object
Average Rating                    float64
Total Feedback                      int64
dtype: object

In [8]:
df_agent_performance['Week']=df_agent_performance['Date'].dt.isocalendar().week

In [9]:
df_agent_performance.head()

,SL No,Date,Agent Name,Total Chats,Average Response Time,Average Resolution Time,Average Rating,Total Feedback,Week
0,1,2022-07-30,Prerna Singh,11,00:00:38,00:04:20,4.11,9,30
1,2,2022-07-30,Nandani Gupta,11,00:01:15,00:28:25,3.14,7,30
2,3,2022-07-30,Ameya Jain,14,00:00:30,00:11:36,4.55,11,30
3,4,2022-07-30,Mahesh Sarade,14,00:01:04,00:15:46,4.71,7,30
4,5,2022-07-30,Swati,14,00:01:11,00:16:33,3.67,6,30


In [10]:
df_agent_performance.Week.unique()

<IntegerArray>
[30, 29, 28, 27, 26]
Length: 5, dtype: UInt32

In [11]:
def weekly_avg_rating(df_agent_performance):
    agents=[name for name in df_agent_performance['Agent Name'].unique()]
    df = pd.DataFrame()
    for agent in agents:
        tmp_df = df_agent_performance[df_agent_performance['Agent Name']==agent]
        # remove the days where the agent didn't have any chat
        tmp_df.drop(tmp_df[tmp_df['Total Chats']==0].index,axis=0,inplace=True)
        warnings.filterwarnings("ignore")
        week_data=tmp_df.groupby(['Week'])['Average Rating'].sum() /tmp_df.groupby(['Week'])['Average Rating'].count()
        week_data=pd.Series(week_data,name=agent)
        df=df.append(week_data)
        df.fillna(value=0,inplace=True)
    return df

In [12]:
agent_performance_by_week=weekly_avg_rating(df_agent_performance)

C:\Users\DELL\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [13]:
agent_performance_by_week

,26,27,28,29,30
Prerna Singh,4.240000,4.340000,4.662000,4.448000,4.250000
Nandani Gupta,4.115000,4.438000,4.218000,4.208000,3.790000
Ameya Jain,0.000000,4.000000,4.256000,4.464000,4.747500
Mahesh Sarade,0.000000,3.626667,3.905000,4.154000,4.232500
Swati,4.485000,4.484000,4.144000,4.156667,4.065000
...,...,...,...,...,...
Boktiar Ahmed Bappy,4.720000,4.610000,4.462500,3.792857,3.470000
Shivananda Sonwane,4.753333,4.426667,4.398333,4.465714,4.751667
Ishawant Kumar,0.000000,4.710000,4.244286,4.215714,4.706667
Deepranjan Gupta,4.230000,4.220000,4.587500,2.750000,4.615000


### 2. Total working days for each agent

In [14]:
df_agent_performance.head()

,SL No,Date,Agent Name,Total Chats,Average Response Time,Average Resolution Time,Average Rating,Total Feedback,Week
0,1,2022-07-30,Prerna Singh,11,00:00:38,00:04:20,4.11,9,30
1,2,2022-07-30,Nandani Gupta,11,00:01:15,00:28:25,3.14,7,30
2,3,2022-07-30,Ameya Jain,14,00:00:30,00:11:36,4.55,11,30
3,4,2022-07-30,Mahesh Sarade,14,00:01:04,00:15:46,4.71,7,30
4,5,2022-07-30,Swati,14,00:01:11,00:16:33,3.67,6,30


In [15]:
len(df_agent_performance[df_agent_performance['Agent Name']=='Ameya Jain'].Date.unique())

30

In [19]:
def total_working_days(df):
    agents = [agent for agent in df['Agent Name'].unique()]
    df2 = pd.DataFrame()
    for agent in agents:
        num = len(df[df['Agent Name']==agent].Date.unique())
        d = pd.Series({'Name':agent, 'Total_working_days':num})
        df2=df2.append(d,ignore_index=True)
    return df2

In [21]:
df_total_working = total_working_days(df_agent_performance)
df_total_working

,Name,Total_working_days
0,Prerna Singh,30.0
1,Nandani Gupta,30.0
2,Ameya Jain,30.0
3,Mahesh Sarade,30.0
4,Swati,30.0
...,...,...
65,Boktiar Ahmed Bappy,30.0
66,Shivananda Sonwane,30.0
67,Ishawant Kumar,30.0
68,Deepranjan Gupta,30.0


### 3. Total query taken

In [24]:
df_agent_performance.groupby('Agent Name')['Total Chats'].sum()

Agent Name
Abhishek           0
Aditya             0
Aditya Shinde    277
Aditya_iot       231
Amersh             0
                ... 
Uday Mishra        0
Vasanth P          0
Vivek             44
Wasim            433
Zeeshan          542
Name: Total Chats, Length: 70, dtype: int64

### 4. Total feedback received

In [30]:
df_agent_performance.groupby('Agent Name')['Total Feedback'].sum()

Agent Name
Abhishek           0
Aditya             0
Aditya Shinde    153
Aditya_iot       131
Amersh             0
                ... 
Uday Mishra        0
Vasanth P          0
Vivek             20
Wasim            284
Zeeshan          335
Name: Total Feedback, Length: 70, dtype: int64

### 5. Agent name with avg rating between 3.5 to 4

In [34]:
df_agent_performance[(df_agent_performance['Average Rating'] >= 3.5) & (df_agent_performance['Average Rating'] <= 4)]['Agent Name']

4                    Swati 
10             Manjunatha A
73      Boktiar Ahmed Bappy
76            Prateek _iot 
78            Nandani Gupta
               ...         
1941         Jawala Prakash
2018          Sanjeev Kumar
2021          Aditya Shinde
2023       Deepranjan Gupta
2097          Sanjeev Kumar
Name: Agent Name, Length: 114, dtype: object

### 6 . Agent with rating less than 3.5

In [36]:
df_agent_performance[(df_agent_performance['Average Rating'] <= 3.5)]['Agent Name']

1            Nandani Gupta
19        Hitesh Choudhary
20              Sanjeevan 
21                Anirudh 
22        Shiva Srivastava
               ...        
2155     Sowmiya Sivakumar
2156               Nitin M
2157                Vivek 
2158         Ayushi Mishra
2159    Chaitra K Hiremath
Name: Agent Name, Length: 1477, dtype: object

### 7. Agent with rating > 4.5

In [37]:
df_agent_performance[(df_agent_performance['Average Rating'] >= 4.5)]['Agent Name']

2               Ameya Jain
3            Mahesh Sarade
5                  Mukesh 
6          Saikumarreddy N
7            Sanjeev Kumar
               ...        
2100        Shubham Sharma
2101         Jaydeep Dixit
2102    Shivananda Sonwane
2103        Khushboo Priya
2104       Hrisikesh Neogi
Name: Agent Name, Length: 325, dtype: object

### 8. Feedback received by agents with rating > 4.5

In [39]:
df_agent_performance[(df_agent_performance['Average Rating'] >= 4.5)].groupby(['Agent Name'])['Total Feedback'].sum()

Agent Name
Aditya Shinde              73
Aditya_iot                 43
Ameya Jain                150
Anirudh                     2
Ankitjha                    1
Aravind                   198
Ayushi Mishra              75
Bharath                   231
Boktiar Ahmed Bappy        78
Chaitra K Hiremath          5
Deepranjan Gupta           83
Harikrishnan Shaji         92
Hrisikesh Neogi           183
Ishawant Kumar             81
Jawala Prakash             43
Jayant Kumar                7
Jaydeep Dixit             179
Khushboo Priya            144
Madhulika G                81
Mahesh Sarade              50
Maitry                     81
Manjunatha A              132
Mithun S                   93
Mukesh                     17
Mukesh Rao                  1
Muskan Garg                14
Nandani Gupta              91
Nishtha Jain               30
Prabir Kumar Satapathy     53
Prateek _iot               54
Prerna Singh              104
Rishav Dash                37
Saikumarreddy N           184

### 9. Avg weekly response time for each agent

In [52]:
df_agent_performance['Average Response Time']=pd.to_datetime(df_agent_performance['Average Response Time'])

In [59]:
df = df_agent_performance['Average Response Time']

In [44]:
df_agent_performance.groupby(['Agent Name']).groupby(['Week'])['Average Response Time']

,SL No,Date,Agent Name,Total Chats,Average Response Time,Average Resolution Time,Average Rating,Total Feedback,Week
0,1,2022-07-30,Prerna Singh,11,00:00:38,00:04:20,4.11,9,30
1,2,2022-07-30,Nandani Gupta,11,00:01:15,00:28:25,3.14,7,30
2,3,2022-07-30,Ameya Jain,14,00:00:30,00:11:36,4.55,11,30
3,4,2022-07-30,Mahesh Sarade,14,00:01:04,00:15:46,4.71,7,30
4,5,2022-07-30,Swati,14,00:01:11,00:16:33,3.67,6,30
...,...,...,...,...,...,...,...,...,...
2155,2156,2022-07-01,Sowmiya Sivakumar,0,00:00:00,00:00:00,0.00,0,26
2156,2157,2022-07-01,Nitin M,0,00:00:00,00:00:00,0.00,0,26
2157,2158,2022-07-01,Vivek,0,00:00:00,00:00:00,0.00,0,26
2158,2159,2022-07-01,Ayushi Mishra,0,00:00:00,00:00:00,0.00,0,26


In [68]:
from datetime import datetime
times = []
for i in range(df_agent_performance.shape[0]):
    d = datetime.strptime(str(df[i]), '%Y-%m-%d %H:%M:%S')
    hr, minu, sec = d.hour, d.minute, d.second
    total_time = hr*3600 + minu*60 + sec
    times.append(total_time)
df_agent_performance['Avg_response_time_in_sec'] = times

In [69]:
df_agent_performance

,SL No,Date,Agent Name,Total Chats,Average Response Time,Average Resolution Time,Average Rating,Total Feedback,Week,Avg_response_time_in_sec
0,1,2022-07-30,Prerna Singh,11,2022-07-30 00:00:38,00:04:20,4.11,9,30,38
1,2,2022-07-30,Nandani Gupta,11,2022-07-30 00:01:15,00:28:25,3.14,7,30,75
2,3,2022-07-30,Ameya Jain,14,2022-07-30 00:00:30,00:11:36,4.55,11,30,30
3,4,2022-07-30,Mahesh Sarade,14,2022-07-30 00:01:04,00:15:46,4.71,7,30,64
4,5,2022-07-30,Swati,14,2022-07-30 00:01:11,00:16:33,3.67,6,30,71
...,...,...,...,...,...,...,...,...,...,...
2155,2156,2022-07-01,Sowmiya Sivakumar,0,2022-07-30 00:00:00,00:00:00,0.00,0,26,0
2156,2157,2022-07-01,Nitin M,0,2022-07-30 00:00:00,00:00:00,0.00,0,26,0
2157,2158,2022-07-01,Vivek,0,2022-07-30 00:00:00,00:00:00,0.00,0,26,0
2158,2159,2022-07-01,Ayushi Mishra,0,2022-07-30 00:00:00,00:00:00,0.00,0,26,0


In [72]:
agents = [agent for agent in df_agent_performance['Agent Name'].unique()]

In [104]:
def wkly_resp_time(df):
    df2 = pd.DataFrame()
    for agent in agents:
        tmp = df[df['Agent Name']==agent]
        tmp.drop(tmp[tmp['Total Chats']==0].index,axis=0,inplace=True)
        warnings.filterwarnings("ignore")
        data=tmp.groupby(['Week'])['Avg_response_time_in_sec'].sum() /tmp.groupby(['Week'])['Avg_response_time_in_sec'].count()
        data=pd.Series(data,name=agent)
        df2=df2.append(data)
        

    return df2

In [105]:
dff = wkly_resp_time(df_agent_performance)
dff

,26,27,28,29,30
Prerna Singh,97.000000,69.000000,58.600000,60.800000,58.800000
Nandani Gupta,162.500000,81.000000,71.000000,80.600000,77.000000
Ameya Jain,NaN,0.000000,45.600000,41.600000,49.500000
Mahesh Sarade,NaN,90.000000,64.666667,82.600000,80.500000
Swati,160.500000,97.400000,108.000000,70.666667,87.000000
...,...,...,...,...,...
Boktiar Ahmed Bappy,60.333333,50.000000,103.250000,66.857143,103.166667
Shivananda Sonwane,77.000000,61.333333,57.333333,59.571429,53.333333
Ishawant Kumar,NaN,69.500000,65.285714,62.714286,55.000000
Deepranjan Gupta,67.000000,83.200000,77.000000,81.000000,69.000000


### 10. Avg weekly response time for each agent


In [87]:
df_agent_performance

,SL No,Date,Agent Name,Total Chats,Average Response Time,Average Resolution Time,Average Rating,Total Feedback,Week,Avg_response_time_in_sec
0,1,2022-07-30,Prerna Singh,11,2022-07-30 00:00:38,00:04:20,4.11,9,30,38
1,2,2022-07-30,Nandani Gupta,11,2022-07-30 00:01:15,00:28:25,3.14,7,30,75
2,3,2022-07-30,Ameya Jain,14,2022-07-30 00:00:30,00:11:36,4.55,11,30,30
3,4,2022-07-30,Mahesh Sarade,14,2022-07-30 00:01:04,00:15:46,4.71,7,30,64
4,5,2022-07-30,Swati,14,2022-07-30 00:01:11,00:16:33,3.67,6,30,71
...,...,...,...,...,...,...,...,...,...,...
2155,2156,2022-07-01,Sowmiya Sivakumar,0,2022-07-30 00:00:00,00:00:00,0.00,0,26,0
2156,2157,2022-07-01,Nitin M,0,2022-07-30 00:00:00,00:00:00,0.00,0,26,0
2157,2158,2022-07-01,Vivek,0,2022-07-30 00:00:00,00:00:00,0.00,0,26,0
2158,2159,2022-07-01,Ayushi Mishra,0,2022-07-30 00:00:00,00:00:00,0.00,0,26,0


In [90]:
df_agent_performance['Average Resolution Time']=pd.to_datetime(df_agent_performance['Average Resolution Time'])

In [91]:
df = df_agent_performance['Average Resolution Time']

In [92]:
from datetime import datetime
times = []
for i in range(df_agent_performance.shape[0]):
    d = datetime.strptime(str(df[i]), '%Y-%m-%d %H:%M:%S')
    hr, minu, sec = d.hour, d.minute, d.second
    total_time = hr*3600 + minu*60 + sec
    times.append(total_time)
df_agent_performance['Avg_resolution_time_in_sec'] = times

In [93]:
df_agent_performance

,SL No,Date,Agent Name,Total Chats,Average Response Time,Average Resolution Time,Average Rating,Total Feedback,Week,Avg_response_time_in_sec,Avg_resolution_time_in_sec
0,1,2022-07-30,Prerna Singh,11,2022-07-30 00:00:38,2022-07-30 00:04:20,4.11,9,30,38,260
1,2,2022-07-30,Nandani Gupta,11,2022-07-30 00:01:15,2022-07-30 00:28:25,3.14,7,30,75,1705
2,3,2022-07-30,Ameya Jain,14,2022-07-30 00:00:30,2022-07-30 00:11:36,4.55,11,30,30,696
3,4,2022-07-30,Mahesh Sarade,14,2022-07-30 00:01:04,2022-07-30 00:15:46,4.71,7,30,64,946
4,5,2022-07-30,Swati,14,2022-07-30 00:01:11,2022-07-30 00:16:33,3.67,6,30,71,993
...,...,...,...,...,...,...,...,...,...,...,...
2155,2156,2022-07-01,Sowmiya Sivakumar,0,2022-07-30 00:00:00,2022-07-30 00:00:00,0.00,0,26,0,0
2156,2157,2022-07-01,Nitin M,0,2022-07-30 00:00:00,2022-07-30 00:00:00,0.00,0,26,0,0
2157,2158,2022-07-01,Vivek,0,2022-07-30 00:00:00,2022-07-30 00:00:00,0.00,0,26,0,0
2158,2159,2022-07-01,Ayushi Mishra,0,2022-07-30 00:00:00,2022-07-30 00:00:00,0.00,0,26,0,0


In [102]:
def wkly_resln_time(df):
    df2 = pd.DataFrame()
    for agent in agents:
        tmp = df[df['Agent Name']==agent]
        tmp.drop(tmp[tmp['Total Chats']==0].index,axis=0,inplace=True)
        warnings.filterwarnings("ignore")
        data=tmp.groupby(['Week'])['Avg_resolution_time_in_sec'].sum() /tmp.groupby(['Week'])['Avg_resolution_time_in_sec'].count()
        data=pd.Series(data,name=agent)
        df2=df2.append(data)
        

    return df2

In [103]:
dff = wkly_resln_time(df_agent_performance)
dff

,26,27,28,29,30
Prerna Singh,928.000000,1150.400000,1968.000000,1237.000000,1221.800000
Nandani Gupta,1575.500000,1402.800000,1430.400000,1556.600000,1897.750000
Ameya Jain,NaN,107.000000,677.200000,697.000000,719.250000
Mahesh Sarade,NaN,846.000000,718.500000,1100.800000,1033.000000
Swati,1589.000000,1236.600000,1444.400000,1094.333333,1115.500000
...,...,...,...,...,...
Boktiar Ahmed Bappy,900.333333,2320.166667,714.250000,695.714286,1061.000000
Shivananda Sonwane,1334.000000,1462.500000,1365.166667,1230.285714,1413.000000
Ishawant Kumar,NaN,1490.750000,794.714286,1079.428571,1145.166667
Deepranjan Gupta,1812.000000,2167.600000,1736.000000,2310.000000,1148.166667


### 11. List of all agents name

In [106]:
agents = [agent for agent in df_agent_performance['Agent Name'].unique()]

In [107]:
agents

['Prerna Singh',
 'Nandani Gupta',
 'Ameya Jain',
 'Mahesh Sarade',
 'Swati ',
 'Mukesh ',
 'Saikumarreddy N',
 'Sanjeev Kumar',
 'Shubham Sharma',
 'Nishtha Jain',
 'Manjunatha A',
 'Khushboo Priya',
 'Suraj S Bilgi',
 'Harikrishnan Shaji',
 'Hrisikesh Neogi',
 'Shivan K',
 'Sowmiya Sivakumar',
 'Madhulika G',
 'Mithun S',
 'Hitesh Choudhary',
 'Sanjeevan ',
 'Anirudh ',
 'Shiva Srivastava',
 'Dibyanshu ',
 'Ashish ',
 'Uday Mishra',
 'Aditya Shinde',
 'Jayant Kumar',
 'Aditya_iot ',
 'Prabir Kumar Satapathy',
 'Saurabh Shukla',
 'Wasim ',
 'Samprit ',
 'Maitry ',
 'Abhishek ',
 'Rishav Dash',
 'Aravind ',
 'Tarun ',
 'Muskan Garg',
 'Ankit Sharma',
 'Bharath ',
 'Ashad Nasim',
 'Vasanth P',
 'Aditya ',
 'Amersh ',
 'Sandipan Saha',
 'Ineuron Intelligence ',
 'Mahak ',
 'Mukesh Rao ',
 'Zeeshan ',
 'Anurag Tiwari',
 'Ankitjha ',
 'Saif Khan',
 'Jaydeep Dixit',
 'Sudhanshu Kumar',
 'Maneesh ',
 'Hyder Abbas',
 'Rohan ',
 'Vivek ',
 'Nitin M',
 'Shivan_S ',
 'Spuri ',
 'Ayushi Mishra',


### 12. Percentage of chat for which they have received feedback

In [108]:
df_agent_performance

,SL No,Date,Agent Name,Total Chats,Average Response Time,Average Resolution Time,Average Rating,Total Feedback,Week,Avg_response_time_in_sec,Avg_resolution_time_in_sec
0,1,2022-07-30,Prerna Singh,11,2022-07-30 00:00:38,2022-07-30 00:04:20,4.11,9,30,38,260
1,2,2022-07-30,Nandani Gupta,11,2022-07-30 00:01:15,2022-07-30 00:28:25,3.14,7,30,75,1705
2,3,2022-07-30,Ameya Jain,14,2022-07-30 00:00:30,2022-07-30 00:11:36,4.55,11,30,30,696
3,4,2022-07-30,Mahesh Sarade,14,2022-07-30 00:01:04,2022-07-30 00:15:46,4.71,7,30,64,946
4,5,2022-07-30,Swati,14,2022-07-30 00:01:11,2022-07-30 00:16:33,3.67,6,30,71,993
...,...,...,...,...,...,...,...,...,...,...,...
2155,2156,2022-07-01,Sowmiya Sivakumar,0,2022-07-30 00:00:00,2022-07-30 00:00:00,0.00,0,26,0,0
2156,2157,2022-07-01,Nitin M,0,2022-07-30 00:00:00,2022-07-30 00:00:00,0.00,0,26,0,0
2157,2158,2022-07-01,Vivek,0,2022-07-30 00:00:00,2022-07-30 00:00:00,0.00,0,26,0,0
2158,2159,2022-07-01,Ayushi Mishra,0,2022-07-30 00:00:00,2022-07-30 00:00:00,0.00,0,26,0,0


In [114]:
df_agent_performance['Feedback_percentage'] = ((df_agent_performance['Total Chats']-df_agent_performance['Total Feedback'])/df_agent_performance['Total Chats'])*100
df_agent_performance['Feedback_percentage']=df_agent_performance['Feedback_percentage'].fillna(value=0)

In [115]:
df_agent_performance[['Agent Name','Feedback_percentage']]

,Agent Name,Feedback_percentage
0,Prerna Singh,18.181818
1,Nandani Gupta,36.363636
2,Ameya Jain,21.428571
3,Mahesh Sarade,50.000000
4,Swati,57.142857
...,...,...
2155,Sowmiya Sivakumar,0.000000
2156,Nitin M,0.000000
2157,Vivek,0.000000
2158,Ayushi Mishra,0.000000


### 13. Total contribution hour for each agent weekly

In [117]:
df=df_agent_report
df

,SL No,Agent,Date,Login Time,Logout Time,Duration
0,1,Shivananda Sonwane,30-Jul-22,03:35:29 PM,05:39:39 PM,02:04:10
1,2,Khushboo Priya,30-Jul-22,03:06:59 PM,03:07:16 PM,00:00:17
2,3,Nandani Gupta,30-Jul-22,03:04:24 PM,05:31:07 PM,02:26:42
3,4,Hrisikesh Neogi,30-Jul-22,02:34:29 PM,03:19:35 PM,00:45:06
4,5,Mukesh,30-Jul-22,02:03:15 PM,03:11:52 PM,01:08:36
...,...,...,...,...,...,...
995,996,Manjunatha A,20-Jul-22,09:03:51 AM,03:02:28 PM,05:58:37
996,997,Bharath,20-Jul-22,09:00:49 AM,03:01:32 PM,06:00:43
997,998,Khushboo Priya,20-Jul-22,08:59:20 AM,02:26:55 PM,05:27:34
998,999,Nishtha Jain,20-Jul-22,08:43:55 AM,02:00:33 PM,05:16:38


In [120]:
df['Date']=pd.to_datetime(df['Date'])
df

,SL No,Agent,Date,Login Time,Logout Time,Duration
0,1,Shivananda Sonwane,2022-07-30,03:35:29 PM,05:39:39 PM,02:04:10
1,2,Khushboo Priya,2022-07-30,03:06:59 PM,03:07:16 PM,00:00:17
2,3,Nandani Gupta,2022-07-30,03:04:24 PM,05:31:07 PM,02:26:42
3,4,Hrisikesh Neogi,2022-07-30,02:34:29 PM,03:19:35 PM,00:45:06
4,5,Mukesh,2022-07-30,02:03:15 PM,03:11:52 PM,01:08:36
...,...,...,...,...,...,...
995,996,Manjunatha A,2022-07-20,09:03:51 AM,03:02:28 PM,05:58:37
996,997,Bharath,2022-07-20,09:00:49 AM,03:01:32 PM,06:00:43
997,998,Khushboo Priya,2022-07-20,08:59:20 AM,02:26:55 PM,05:27:34
998,999,Nishtha Jain,2022-07-20,08:43:55 AM,02:00:33 PM,05:16:38


In [129]:
df['Login Time']=pd.to_datetime(df['Login Time'])
df

,SL No,Agent,Date,Login Time,Logout Time,Duration
0,1,Shivananda Sonwane,2022-07-30,2022-07-31 15:35:29,05:39:39 PM,02:04:10
1,2,Khushboo Priya,2022-07-30,2022-07-31 15:06:59,03:07:16 PM,00:00:17
2,3,Nandani Gupta,2022-07-30,2022-07-31 15:04:24,05:31:07 PM,02:26:42
3,4,Hrisikesh Neogi,2022-07-30,2022-07-31 14:34:29,03:19:35 PM,00:45:06
4,5,Mukesh,2022-07-30,2022-07-31 14:03:15,03:11:52 PM,01:08:36
...,...,...,...,...,...,...
995,996,Manjunatha A,2022-07-20,2022-07-31 09:03:51,03:02:28 PM,05:58:37
996,997,Bharath,2022-07-20,2022-07-31 09:00:49,03:01:32 PM,06:00:43
997,998,Khushboo Priya,2022-07-20,2022-07-31 08:59:20,02:26:55 PM,05:27:34
998,999,Nishtha Jain,2022-07-20,2022-07-31 08:43:55,02:00:33 PM,05:16:38


In [130]:
df['Logout Time']=pd.to_datetime(df['Logout Time'])
df

,SL No,Agent,Date,Login Time,Logout Time,Duration
0,1,Shivananda Sonwane,2022-07-30,2022-07-31 15:35:29,2022-07-31 17:39:39,02:04:10
1,2,Khushboo Priya,2022-07-30,2022-07-31 15:06:59,2022-07-31 15:07:16,00:00:17
2,3,Nandani Gupta,2022-07-30,2022-07-31 15:04:24,2022-07-31 17:31:07,02:26:42
3,4,Hrisikesh Neogi,2022-07-30,2022-07-31 14:34:29,2022-07-31 15:19:35,00:45:06
4,5,Mukesh,2022-07-30,2022-07-31 14:03:15,2022-07-31 15:11:52,01:08:36
...,...,...,...,...,...,...
995,996,Manjunatha A,2022-07-20,2022-07-31 09:03:51,2022-07-31 15:02:28,05:58:37
996,997,Bharath,2022-07-20,2022-07-31 09:00:49,2022-07-31 15:01:32,06:00:43
997,998,Khushboo Priya,2022-07-20,2022-07-31 08:59:20,2022-07-31 14:26:55,05:27:34
998,999,Nishtha Jain,2022-07-20,2022-07-31 08:43:55,2022-07-31 14:00:33,05:16:38


In [124]:
for i in range(len(df['Duration'])):
    j = df['Duration'][i].split(':')
    if int(j[0]) > 23:
        print(j)
        print(i)

['24', '09', '54']
557
['514', '08', '20']
999


In [128]:
df.loc[557,:]

SL No                          558
Agent                Jaydeep Dixit
Date           2022-07-24 00:00:00
Login Time             08:52:20 PM
Logout Time            09:02:14 PM
Duration                  24:09:54
Name: 557, dtype: object

In [137]:
t1=df['Login Time'][0]
t2=df['Logout Time'][0]

In [140]:
dd1 = datetime.strptime(str(t1), '%Y-%m-%d %H:%M:%S')
dd2 = datetime.strptime(str(t2), '%Y-%m-%d %H:%M:%S')
dur = (dd2.hour-dd1.hour)*3600+(dd2.minute-dd1.minute)*60 + dd2.second-dd1.second
dur

7450

In [142]:
def dur_sec(df):
    l = []
    for i in range(len(df)):
        t1=df['Login Time'][i]
        t2=df['Logout Time'][i]
        dd1 = datetime.strptime(str(t1), '%Y-%m-%d %H:%M:%S')
        dd2 = datetime.strptime(str(t2), '%Y-%m-%d %H:%M:%S')
        dur = (dd2.hour-dd1.hour)*3600+(dd2.minute-dd1.minute)*60 + dd2.second-dd1.second
        l.append(dur)
    df['Duration_in_sec']=l
    return df

In [143]:
df1 = dur_sec(df)
df1

,SL No,Agent,Date,Login Time,Logout Time,Duration,Duration_in_sec
0,1,Shivananda Sonwane,2022-07-30,2022-07-31 15:35:29,2022-07-31 17:39:39,02:04:10,7450
1,2,Khushboo Priya,2022-07-30,2022-07-31 15:06:59,2022-07-31 15:07:16,00:00:17,17
2,3,Nandani Gupta,2022-07-30,2022-07-31 15:04:24,2022-07-31 17:31:07,02:26:42,8803
3,4,Hrisikesh Neogi,2022-07-30,2022-07-31 14:34:29,2022-07-31 15:19:35,00:45:06,2706
4,5,Mukesh,2022-07-30,2022-07-31 14:03:15,2022-07-31 15:11:52,01:08:36,4117
...,...,...,...,...,...,...,...
995,996,Manjunatha A,2022-07-20,2022-07-31 09:03:51,2022-07-31 15:02:28,05:58:37,21517
996,997,Bharath,2022-07-20,2022-07-31 09:00:49,2022-07-31 15:01:32,06:00:43,21643
997,998,Khushboo Priya,2022-07-20,2022-07-31 08:59:20,2022-07-31 14:26:55,05:27:34,19655
998,999,Nishtha Jain,2022-07-20,2022-07-31 08:43:55,2022-07-31 14:00:33,05:16:38,18998


In [144]:
df1['Duration_hr'] = df1['Duration_in_sec'] / 3600
df1

,SL No,Agent,Date,Login Time,Logout Time,Duration,Duration_in_sec,Duration_hr
0,1,Shivananda Sonwane,2022-07-30,2022-07-31 15:35:29,2022-07-31 17:39:39,02:04:10,7450,2.069444
1,2,Khushboo Priya,2022-07-30,2022-07-31 15:06:59,2022-07-31 15:07:16,00:00:17,17,0.004722
2,3,Nandani Gupta,2022-07-30,2022-07-31 15:04:24,2022-07-31 17:31:07,02:26:42,8803,2.445278
3,4,Hrisikesh Neogi,2022-07-30,2022-07-31 14:34:29,2022-07-31 15:19:35,00:45:06,2706,0.751667
4,5,Mukesh,2022-07-30,2022-07-31 14:03:15,2022-07-31 15:11:52,01:08:36,4117,1.143611
...,...,...,...,...,...,...,...,...
995,996,Manjunatha A,2022-07-20,2022-07-31 09:03:51,2022-07-31 15:02:28,05:58:37,21517,5.976944
996,997,Bharath,2022-07-20,2022-07-31 09:00:49,2022-07-31 15:01:32,06:00:43,21643,6.011944
997,998,Khushboo Priya,2022-07-20,2022-07-31 08:59:20,2022-07-31 14:26:55,05:27:34,19655,5.459722
998,999,Nishtha Jain,2022-07-20,2022-07-31 08:43:55,2022-07-31 14:00:33,05:16:38,18998,5.277222


In [146]:
df1['Week']=df1['Date'].dt.isocalendar().week
df1

,SL No,Agent,Date,Login Time,Logout Time,Duration,Duration_in_sec,Duration_hr,Week
0,1,Shivananda Sonwane,2022-07-30,2022-07-31 15:35:29,2022-07-31 17:39:39,02:04:10,7450,2.069444,30
1,2,Khushboo Priya,2022-07-30,2022-07-31 15:06:59,2022-07-31 15:07:16,00:00:17,17,0.004722,30
2,3,Nandani Gupta,2022-07-30,2022-07-31 15:04:24,2022-07-31 17:31:07,02:26:42,8803,2.445278,30
3,4,Hrisikesh Neogi,2022-07-30,2022-07-31 14:34:29,2022-07-31 15:19:35,00:45:06,2706,0.751667,30
4,5,Mukesh,2022-07-30,2022-07-31 14:03:15,2022-07-31 15:11:52,01:08:36,4117,1.143611,30
...,...,...,...,...,...,...,...,...,...
995,996,Manjunatha A,2022-07-20,2022-07-31 09:03:51,2022-07-31 15:02:28,05:58:37,21517,5.976944,29
996,997,Bharath,2022-07-20,2022-07-31 09:00:49,2022-07-31 15:01:32,06:00:43,21643,6.011944,29
997,998,Khushboo Priya,2022-07-20,2022-07-31 08:59:20,2022-07-31 14:26:55,05:27:34,19655,5.459722,29
998,999,Nishtha Jain,2022-07-20,2022-07-31 08:43:55,2022-07-31 14:00:33,05:16:38,18998,5.277222,29


In [148]:
def wkly_cont_time(df):
    agents = [agent for agent in df['Agent'].unique()]
    df2 = pd.DataFrame()
    for agent in agents:
        tmp = df[df['Agent']==agent]
        data=tmp.groupby(['Week'])['Duration_hr'].sum()
        data=pd.Series(data,name=agent)
        df2=df2.append(data)
        

    return df2

In [149]:
dff = wkly_cont_time(df1)
dff

,29,30,26
Shivananda Sonwane,-75.164722,-67.544444,NaN
Khushboo Priya,21.717500,21.844167,NaN
Nandani Gupta,17.334167,22.839444,NaN
Hrisikesh Neogi,2.893611,6.680833,NaN
Mukesh,NaN,8.905556,NaN
Sowmiya Sivakumar,17.066389,27.689722,NaN
Manjunatha A,18.351667,22.924722,NaN
Harikrishnan Shaji,21.438889,32.278611,NaN
Suraj S Bilgi,NaN,12.560000,NaN
Shivan K,16.714444,19.391944,NaN
